In [15]:
!pip install moviepy==1.0.3 "scikit-video>=1.1.11" numpy==1.26.0 'opencv-python>=4.10.0.84' 'diffusers>=0.30.1'

# 需要重启

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
DEPRECATION: omegaconf 2.0.5 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import sys
sys.path.append('/mnt/ceph/develop/jiawei/lora_dataset/Keyframe-Extraction-for-video-summarization/src')

from rife_model import load_rife_model, rife_inference_with_latents
from huggingface_hub import hf_hub_download, snapshot_download
import torch
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

/mnt/ceph/develop/jiawei/conda_env/keyframe_extra/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
snapshot_download(repo_id="AlexWortega/RIFE", local_dir="model_rife")


KeyboardInterrupt: 

### 读取视频帧，做一倍插针

In [7]:
import torch
torch.set_grad_enabled(False)
if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

frame_interpolation_model = load_rife_model("model_rife")

frame_interpolation_model.device()


In [8]:
import cv2
import numpy as np
import utils


def loadvideo_pt(video_path):
    
    video_capture = cv2.VideoCapture(video_path)
    tot_frame = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total frames
    pt_frame_data = []
    
    # Read each frame using VideoCapture
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break
        # BGR to RGB
        frame_rgb = frame[..., ::-1]
        frame_rgb = frame_rgb.copy()
        tensor = torch.from_numpy(frame_rgb).float().to("cpu", non_blocking=True).float() / 255.0
        pt_frame_data.append(
            tensor.permute(2, 0, 1)
        )  # to [c, h, w,]
    
    video_capture.release()  # Release the video after done
    
    pt_frame = torch.from_numpy(np.stack(pt_frame_data))
    pt_frame = pt_frame.to(device)
    pt_frame = pt_frame.unsqueeze(0)
    return pt_frame.to(dtype=torch.float16)

In [9]:
from diffusers.image_processor import VaeImageProcessor
from diffusers.utils import export_to_video
import math

def rile_with_path(video_path, export_video_path):
    pt_frame = loadvideo_pt(video_path)
    rife_pt_frame = rife_inference_with_latents(frame_interpolation_model,pt_frame )
    
    pt_image = rife_pt_frame[0]
    pt_image = torch.stack([pt_image[i] for i in range(pt_image.shape[0])])
     
    image_np = VaeImageProcessor.pt_to_numpy(pt_image)
    image_pil = VaeImageProcessor.numpy_to_pil(image_np)
     
    export_to_video(image_pil, export_video_path, fps=math.ceil((len(image_pil) - 1) / 6))

In [10]:

def load_file_by_extension(file_dir, extension):
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == f'.{extension}': 
                L.append(file) 
        return L 

def rife_video_with_chunk(root_path, video_source):
    save_path = f'{root_path}/{video_source}/' 
 
    scene_csv_path=f'{save_path}/scene/{video_source}_scene_keyframe.csv'
    videos_output_dir = f'{save_path}/scene_chunks/videos'
    labels_output_dir = f'{save_path}/scene_chunks/labels'
    videos_files = load_file_by_extension(videos_output_dir, 'mp4')
    labels_files = load_file_by_extension(labels_output_dir, 'txt')

    print(f"load videos_files:{videos_files}")
    for i, filename in enumerate(videos_files):
        video_path = f'{videos_output_dir}/{filename}'
        export_video_path = f'{videos_output_dir}/rife_{filename}'
        rile_with_path(video_path,export_video_path)
    

    for i, lable_filename in enumerate(labels_files):
        lable_path = f'{videos_output_dir}/{lable_filename}'
        export_lable_path = f'{videos_output_dir}/rife_{lable_filename}'
        shutil.copy(lable_path, export_lable_path)

    


 

In [11]:
rife_video_with_chunk('/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/','test_1')

load videos_files:['chunk_1029_1.mp4', 'chunk_1029_2.mp4', 'chunk_1029_3.mp4', 'chunk_1029_4.mp4', 'chunk_1125_1.mp4', 'chunk_1125_2.mp4', 'chunk_1125_3.mp4', 'chunk_1139_1.mp4', 'chunk_1139_2.mp4', 'chunk_1139_3.mp4', 'chunk_1268_1.mp4', 'chunk_1268_2.mp4', 'chunk_1268_3.mp4', 'chunk_1321_1.mp4', 'chunk_1321_2.mp4', 'chunk_1321_3.mp4', 'chunk_1321_4.mp4', 'chunk_1321_5.mp4', 'chunk_1511_1.mp4', 'chunk_1511_2.mp4', 'chunk_1511_3.mp4', 'chunk_1511_4.mp4', 'chunk_1511_5.mp4', 'chunk_243_1.mp4', 'chunk_243_2.mp4', 'chunk_269_1.mp4', 'chunk_269_2.mp4', 'chunk_293_1.mp4', 'chunk_293_2.mp4', 'chunk_347_1.mp4', 'chunk_347_2.mp4', 'chunk_440_1.mp4', 'chunk_440_2.mp4', 'chunk_542_1.mp4', 'chunk_542_2.mp4', 'chunk_697_1.mp4', 'chunk_697_2.mp4', 'chunk_774_1.mp4', 'chunk_774_2.mp4', 'chunk_774_3.mp4', 'chunk_774_4.mp4', 'chunk_924_1.mp4', 'chunk_924_2.mp4', 'chunk_924_3.mp4', 'chunk_924_4.mp4', 'chunk_984_1.mp4', 'chunk_984_2.mp4', 'chunk_984_3.mp4', 'chunk_984_4.mp4', 'rife_chunk_1029_1.mp4', 'r


RIFE inference:   0%|                                                                                                             | 0/71 [00:00<?, ?it/s]

samples dtype:torch.float16
samples shape:torch.Size([71, 3, 960, 720])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3

RIFE inference:   0%|                                                                                                             | 0/71 [00:01<?, ?it/s]

RIFE inference:   0%|                                                                                                             | 0/71 [00:00<?, ?it/s]

samples dtype:torch.float16
samples shape:torch.Size([71, 3, 960, 720])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3

RIFE inference:   0%|                                                                                                             | 0/71 [00:01<?, ?it/s]

frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])
frame shape:torch.Size([1, 3, 960, 736])
tmp_output[0] shape:torch.Size([1, 3, 960, 736])



RIFE inference:   0%|                                                                                                             | 0/71 [05:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import os
  
root_path = '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/'
for root, dirs, files in os.walk(root_path):
    # 如果你只想获取下一层的子目录，可以在这里筛选
    if root == root_path:
        # root_dir 下的直接子目录就是 dirs 中的项
        for dir in dirs:
            rife_video_with_chunk(root_path,dir)
    